In [ ]:
import geemap
import ee
if not ee.data._credentials: ee.Initialize()

import sys
sys.path.append('../src/geepatches')
import geeutils
import geemask


__aligning roi to reference__

In [ ]:
class Foo(object):
    def __init__(self, eepoint,eedate,eerefimagecollection,refroipixelsdiameter,eetrgimagecollection,trgroipixelsdiameter):
        self.eepoint = eepoint
        self.eerefimagecollection = eerefimagecollection
        self.refroipixelsdiameter = refroipixelsdiameter
        self.eetrgimagecollection = eetrgimagecollection
        self.trgroipixelsdiameter = trgroipixelsdiameter
        #
        # find reference image - assume single band, or all bands having identical projection
        #
        eerefimage = geeutils.someImageNear(eerefimagecollection, eedate, eepoint)
        #
        # find roi center
        #
        refroisize = round(refroipixelsdiameter)                              #  "an integer" I said.
        refroisize = max(refroisize, 1)                                       #  preferably larger then 1
        if (refroisize %2) == 0:                                              #  even diameter
            eeroicenterpoint = geeutils.pixelinterspoint(eepoint, eerefimage) #  roi center on refimage pixels intersection
        else:                                                                 #  odd diameter
            eeroicenterpoint = geeutils.pixelcenterpoint(eepoint, eerefimage) #  roi center on refimage pixel center
        #
        # find actual roi -  roi radius for odd sizes: 1, 2, 3, ... - for even sizes: 0.5, 1.5, 2.5, ...
        #
        eerefroi = geeutils.squarerasterboundsroi(eeroicenterpoint, refroisize/2, eerefimage)
        #
        # find roi origin to translate to
        #
        eerefroiulx = eerefroi.coordinates().flatten().get(0)
        eerefroiuly = eerefroi.coordinates().flatten().get(1)
        #
        # find target image collection
        #
        eeimagecollection = eetrgimagecollection.filterBounds(eerefroi)
        #
        # select target image
        #
        eesrcimage = geeutils.someImageNear(eeimagecollection, eedate, eeroicenterpoint)
        #
        # warp target image to get congruent roi
        #
        eetrgimage = eesrcimage.reproject(eerefimage.projection()
                                  .translate(eerefroiulx, eerefroiuly)
                                  .scale(refroipixelsdiameter/trgroipixelsdiameter, refroipixelsdiameter/trgroipixelsdiameter))
        #
        #
        #
        self.eerefimage       = eerefimage
        self.eeroicenterpoint = eeroicenterpoint
        self.eerefroi         = eerefroi
        self.eesrcimage       = eesrcimage 
        self.eetrgimage       = eetrgimage
        #
        #
        #
        print("ref - pixelcount:",
            eerefimage.select(0).unmask(sameFootprint=False).reduceRegion(ee.Reducer.count(), eerefroi).getInfo())
        print("src - pixelcount:",
            eesrcimage.select(0).unmask(sameFootprint=False).reduceRegion(ee.Reducer.count(), eerefroi).getInfo())
        print("trg - pixelcount:",
            eetrgimage.select(0).unmask(sameFootprint=False).reduceRegion(ee.Reducer.count(), eerefroi).getInfo())


__Proba V 333m with reference S2 10m__

In [ ]:
eepoint              = geeutils.tapspoint
eedate               = geeutils.fleecycloudsday
eerefimagecollection = geeutils.s2ImageCollection.select(['B4', 'B8'])
refroipixelsdiameter = 32
eetrgimagecollection = geeutils.pv333ImageCollection.select(['RED', 'NIR'])
trgroipixelsdiameter = 2

foo = Foo(eepoint,eedate,eerefimagecollection,refroipixelsdiameter,eetrgimagecollection,trgroipixelsdiameter)

eerefimage       = geeutils._ndvi(foo.eerefimage, 'B8', 'B4') # B4~Red B8~Nir
eerefroi         = foo.eerefroi
eeroicenterpoint = foo.eeroicenterpoint
eesrcimage       = geeutils._ndvi(foo.eesrcimage, 'NIR', 'RED')
eetrgimage       = geeutils._ndvi(foo.eetrgimage, 'NIR', 'RED')


map = geemap.Map(height='600px')
map.centerObject(eepoint, 16)
map.addLayer(eesrcimage, {'min':0, 'max':1}, 'eesrcimage')
map.addLayer(eerefimage, {'min':0, 'max':1}, 'eerefimage')
map.addLayer(eetrgimage, {'min':0, 'max':1}, 'eetrgimage')
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 2),         {'palette':'#0000ff'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ffffff'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'eeroicenterpoint')
map


__Sentinel 1 VV/VH bands with reference S2 10m__

In [ ]:
eepoint              = geeutils.bobspoint
eedate               = geeutils.fleecycloudsday
eerefimagecollection = geeutils.s2ImageCollection.select(['B4', 'B8'])
refroipixelsdiameter = 15
eetrgimagecollection = geeutils.s1rbgImageCollection.select(['VV','VH'])
trgroipixelsdiameter = 15

foo = Foo(eepoint,eedate,eerefimagecollection,refroipixelsdiameter,eetrgimagecollection,trgroipixelsdiameter)

eerefimage       = geeutils._ndvi(foo.eerefimage, 'B8', 'B4') # B4~Red B8~Nir
eerefroi         = foo.eerefroi
eeroicenterpoint = foo.eeroicenterpoint
eesrcimage       = geeutils._rvi(foo.eesrcimage)
eetrgimage       = geeutils._rvi(foo.eetrgimage)


map = geemap.Map(height='600px')
map.centerObject(eepoint, 18)
map.addLayer(eesrcimage, {'min':0, 'max':1}, 'eesrcimage')
map.addLayer(eerefimage, {'min':0, 'max':1}, 'eerefimage')
map.addLayer(eetrgimage, {'min':0, 'max':1}, 'eetrgimage')
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 2),         {'palette':'#0000ff'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ff0000'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'eeroicenterpoint')
map

__Sentinel 1 'angle' band with reference S2 10m__

In [ ]:
eepoint              = geeutils.bobspoint
eedate               = geeutils.fleecycloudsday
eerefimagecollection = geeutils.s2ImageCollection.select(['B4', 'B8'])
refroipixelsdiameter = 128
eetrgimagecollection = geeutils.s1rbgImageCollection.select('angle')
trgroipixelsdiameter = 2

foo = Foo(eepoint,eedate,eerefimagecollection,refroipixelsdiameter,eetrgimagecollection,trgroipixelsdiameter)

eerefimage       = geeutils._ndvi(foo.eerefimage, 'B8', 'B4') # B4~Red B8~Nir
eerefroi         = foo.eerefroi
eeroicenterpoint = foo.eeroicenterpoint
eesrcimage       = foo.eesrcimage
eetrgimage       = foo.eetrgimage


map = geemap.Map(height='600px')
map.centerObject(eepoint, 11)
map.addLayer(eesrcimage, {'min':30, 'max':38}, 'eesrcimage')
map.addLayer(eerefimage, {'min':0,  'max':1},  'eerefimage')
map.addLayer(eetrgimage, {'min':30, 'max':38}, 'eetrgimage')
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 2),         {'palette':'#0000ff'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ff0000'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'eeroicenterpoint')
map

__Sentinel 2 SCL bands with reference S2 10m__

In [ ]:
eepoint              = geeutils.tapspoint
eedate               = geeutils.fleecycloudsday
eerefimagecollection = geeutils.s2ImageCollection.select(['B4', 'B8'])
refroipixelsdiameter = 32
eetrgimagecollection = geeutils.s2sclImageCollection
trgroipixelsdiameter = 16

foo = Foo(eepoint,eedate,eerefimagecollection,refroipixelsdiameter,eetrgimagecollection,trgroipixelsdiameter)

eerefimage       = geeutils._ndvi(foo.eerefimage, 'B8', 'B4') # B4~Red B8~Nir
eerefroi         = foo.eerefroi
eeroicenterpoint = foo.eeroicenterpoint
eesrcimage       = foo.eesrcimage
eetrgimage       = foo.eetrgimage


map = geemap.Map(height='600px')
map.centerObject(eepoint, 17)
map.addLayer(eesrcimage, geeutils.s2sclvisParams, 'eesrcimage')
map.addLayer(eerefimage, {'min':0,  'max':1},     'eerefimage')
map.addLayer(eetrgimage, geeutils.s2sclvisParams, 'eetrgimage')
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 2),         {'palette':'#0000ff'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ff0000'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'eeroicenterpoint')
map

__Sentinel 2 SCL ConvMask band with reference S2 10m__

In [ ]:
def convmask(image):
    return ee.Image((geemask.ConvMask( [[2, 4, 5, 6, 7], [3, 8, 9, 10, 11]], [20*9, 20*101] ,[-0.057, 0.025] )
            .makemask(image)
            .rename('MASK')
            .copyProperties(image, ['system:id', 'system:time_start'])))

eepoint              = geeutils.tapspoint
eedate               = geeutils.fleecycloudsday
eerefimagecollection = geeutils.s2ImageCollection.select(['B4', 'B8'])
refroipixelsdiameter = 32
eetrgimagecollection = geeutils.s2sclImageCollection.map(convmask)
trgroipixelsdiameter = 16

foo = Foo(eepoint,eedate,eerefimagecollection,refroipixelsdiameter,eetrgimagecollection,trgroipixelsdiameter)

eerefimage       = geeutils._ndvi(foo.eerefimage, 'B8', 'B4') # B4~Red B8~Nir
eerefroi         = foo.eerefroi
eeroicenterpoint = foo.eeroicenterpoint
eesrcimage       = foo.eesrcimage
eetrgimage       = foo.eetrgimage

eesrcimage = eesrcimage.updateMask(eesrcimage.eq(1))
eetrgimage = eetrgimage.updateMask(eetrgimage.eq(1))

map = geemap.Map(height='600px')
map.centerObject(eepoint, 13)
map.addLayer(eesrcimage, {'min':0,  'max':1},                                          'eesrcimage')
map.addLayer(eerefimage, {'min':0,  'max':1, 'palette':geeutils.ndvivisParamsPalette}, 'eerefimage')
map.addLayer(eetrgimage, {'min':0,  'max':1},                                          'eetrgimage')
map.addLayer(geeutils.outlinegeometryimage(eerefroi, 0, 2),         {'palette':'#0000ff'}, 'eerefroi')
map.addLayer(geeutils.outlinegeometryimage(eepoint,          1, 2), {'palette':'#ff0000'}, 'eepoint')
map.addLayer(geeutils.outlinegeometryimage(eeroicenterpoint, 1, 2), {'palette':'#00ff00'}, 'eeroicenterpoint')
map